### 🐁 AmadeusGPT Quickstart

---


#### Notebook Overview:
1. Bootstrap the Jupyter interpreter 🛠️
2. Load secrets & config 🔑
3. Project initialization 📁
4. Run a demo query 🎬
5. Appendix 📚

#### 1 · Bootstrap the Jupyter interpreter 🛠️
-  Run the below cell *once* per fresh clone or when dependencies change
-  ⚠️ Select a 3.10 Python environment before you run the below code (if you already have the AmadeusGPT Jupyter kernel select that and skip to [Jump to “Load secrets & config”](#load-secrets-config)
)
<a id="bootstrap-jupyter-interpreter"></a>

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet -e .            # editable install → reads pyproject.toml
%pip install --quiet ipykernel       # needed for kernel registration

Register the interpreter as a named Jupyter kernel - unless you already have access to the AmadeusGPT (venv)

In [ ]:
!python -m ipykernel install --user --name amadeus-gpt-env --display-name "AmadeusGPT (venv)"

print("✅  Bootstrapping done – now **Restart** the kernel (Kernel ▶ Restart) and select “AmadeusGPT (venv)” in the top-right kernel picker.")

<a id="load-secrets-config"></a>
#### 2 · Load secrets & config 🔑


In [ ]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path, override=True)
print("🔐  Environment loaded from:", env_path or "❌  .env file not found. Create one and add your OPENAI_API_KEY='api_key' to complete this step")


#### 3 · Project initialisation 📁


In [6]:
import os
from amadeusgpt import AMADEUS, create_project
from amadeusgpt.utils import parse_result

# ── resolve repo root dynamically ──
ROOT = os.getcwd()

DATA_DIR   = os.path.join(ROOT, "data")
RESULT_DIR = os.path.join(ROOT, "video_results")

project_cfg = create_project(
    data_folder   = DATA_DIR,
    result_folder = RESULT_DIR,
    **{
        "data_info.video_suffix": ".avi",
        "keypoint_info.use_3d"  : True
    }
)

amadeus = AMADEUS(project_cfg)
print("✅  Project ready:", project_cfg)


Project created at c:\Users\hugos\Desktop\AmadeusGPT\video_results. Results will be saved to c:\Users\hugos\Desktop\AmadeusGPT\video_results
The project will load video files (*.avi) and optionally keypoint files from c:\Users\hugos\Desktop\AmadeusGPT\data
A copy of the project config file is saved at c:\Users\hugos\Desktop\AmadeusGPT\video_results\config.yaml
{'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data',
               'result_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\video_results',
               'video_suffix': '.avi'},
 'keypoint_info': {'include_confidence': False, 'use_3d': True},
 'llm_info': {'gpt_model': 'gpt-4o',
              'keep_last_n_messages': 2,
              'max_tokens': 4096,
              'temperature': 0.0},
 'object_info': {'load_objects_from_disk': False, 'use_grid_objects': False},
 'video_info': {'scene_frame_number': 1}}
✅  Project ready: {'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data', 'resu

#### 4 · Run a demo query 🎬


In [8]:
query = "Plot the trajectory of the animal using the animal center and color it by time"
msg   = amadeus.step(query)
parse_result(amadeus, msg)          # renders figure / prints response
print("✅  Query executed successfully!")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.0321 $
current input tokens 3015
current accumulated tokens 6090


To plot the trajectory of the animal using the animal center and color it by time, we will follow these steps:

1. Retrieve the keypoints data for the animal.
2. Calculate the center of the animal for each frame by averaging the x, y, and z coordinates of all keypoints.
3. Plot the trajectory of the animal's center and color it by time.

Let's implement this in the code.

✅  Query executed successfully!


### Appendix 📚

---


#### 1. Updating code  
- After you’ve made changes to your package source, simply re-run the dependency‐install cell. [Re-install dependencies here.](#bootstrap-jupyter-interpreter)
#### 2. Troubleshooting

* **Kernel name not visible?**

  * Reload VS Code/Notebook window (e.g. ⌘⇧P → “Developer: Reload Window”).
  * Make sure you're selecting Jupyter Kernels not Python Environments to find `AmadeusGPT (venv)`
  * Ensure the workspace interpreter is set to your `.venv` (click the Python version in the status bar).

* **Version or Downloading Issues?**
  
  * Versioning errors occur if your Kernel is not Python version 3.10 (required for AmadeusGPT).
    *  Ensure that you have this version of Python installed and have selected it as the version for your environment
    *  You can check this in top right corner. If you see it's Python 3.10... that's not the issue.

* **“Package X not found” after reinstalling**

  * Confirm you’ve restarted the kernel (Kernel ▶ Restart).
  * Run `pip list` in a cell to verify that your package is installed in the active environment.

* **Still stuck?**

  * In a terminal, run:

    ```bash
    jupyter kernelspec list
    ```

    to verify that `amadeus-gpt-env` appears under your user kernels.
  * Stop & restart the Jupyter server
    (in VS Code: ⌘⇧P → “Jupyter: Stop Jupyter Server”, then run any cell again).